<a href="https://colab.research.google.com/github/a-cordeiro/Simposio_PO/blob/main/IC_UPIT_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import plotly.express as px
!pip install mip
from mip import *
import IPython.display
import ipywidgets

In [ ]:
IPython.display.Image(url= "https://user-images.githubusercontent.com/92440796/158081607-e9aafe73-e87c-4290-9fd3-046236f10427.png")

<h1 align="center">Mine Planning Optimization <br>UPIT - Marvin Dataset - Python 3.7</h1>

> **Advisor**: Douglas Mazzinghy

> **Students**: <br>
Augusto Yuri Cordeiro Santos  
Fernanda Oliveira Marques  
Hayder Matos Batista Sobrinho  


### Importing the dataset.

In [ ]:
from google.colab import files
uploaded = files.upload()
dataset = pd.read_csv(r'dataset_marvin_UPIT.csv')
dataset_minelib = pd.read_csv(r"marvin_solution.txt")

Saving dataset_marvin_minelib.csv to dataset_marvin_minelib (2).csv
Saving dataset_marvin_UPIT.csv to dataset_marvin_UPIT (3).csv


### Here, we're storing all of the dataset information in variables of type <em> list </em>.

In [ ]:
# Dataset content
ID = list(dataset['id'].values)
X = list(dataset['x'].values)
Y = list(dataset['y'].values)
Z = list(dataset['z'].values)
Tonn = list(dataset['tonn'].values)
Au = list(dataset['au [ppm]'].values)
Cu = list(dataset['cu %'].values)
Profit = list(dataset['proc_profit'].values)

## Model, Variables and Objective
### In this cell, we're creating the optimization model with the Python-MIP package. For this model, we've chosen the CBC solver and established the optimization objetive as maximize, once we're looking for the highest profit in the determined conditions.

In [ ]:
# Model
Model = Model(sense = MAXIMIZE, solver_name=CBC)

# Variables
Mine = [Model.add_var(var_type=BINARY) for i in ID]
Plant = [Model.add_var(var_type=BINARY) for i in ID]

# Objective
Model.objective = maximize(xsum(
    Plant[i] * Profit[i] * Tonn[i] - Mine[i] * 0.9 * Tonn[i] for i in ID))

## Choosing the precedence
### If for a block to be removed the 5 or 9 blocks above have to be removed as well.

In [ ]:
Precedence = ipywidgets.Dropdown(
    options=[5, 9],
    value=5,
    description='Precedence:',
)
Precedence

Dropdown(description='Precedence:', options=(5, 9), value=5)

## Creating constraints

#### At this moment, we're creating both constraints used: the mining/plant constraints and the precedence constraints. Mining/plant constraints define that, for a block to be processed, it has to be extracted first. Precedence constraints, in another way, determine that, for a block to be extracted, all of the nine blocks above it must have been extracted as well. 

In [ ]:
# Mine-Plant constraints
for i in ID:
    Model.add_constr(Mine[i] >= Plant[i])

# Precedence constraints

# If there are 9 precedence blocks
if Precedence.value == 9:

  # Searching "i" for each value in ID, that being the block underneath (inferior)
  for i in ID:

      # Searching "k" in ID, the block above
      for k in ID:

          # If "k" and "i" are different blocks and "k" is below "i"
          if k != i and Z[k] == Z[i] + 1:

              # If the coordinate X of "k" is near the one from "i"
              if X[k] == X[i] -1 or X[k] == X[i] or X[k] == X[i] + 1:

                  # If the coordinate Y of "k" is near the one from "i"
                  if Y[k] == Y[i] -1 or Y[k] == Y[i] or Y[k] == Y[i] + 1:
                      Model.add_constr(Mine[k] >= Mine[i])

# If there are 5 precedence blocks
if Precedence.value == 5:

    # Searching "i" for each value in ID, that being the block underneath (inferior)
    for i in ID:

        # Searching "k" in ID, the block above
        for k in ID:

            # If "k" and "i" are different blocks and "i" is below "k"
            if k != i and Z[i] == Z[k] + 1:

                # If the coordinate X is the same for both and Y is '1' apart
                if X[i] == X[k] and (Y[i] == Y[k] or Y[i] - 1 == Y[k] or Y[i] + 1 == Y[k]):
                    Model.add_constr(Mine[i] >= Mine[k])
                # If the coordinate Y is the same for both and X is '1' apart
                elif Y[i] == Y[k] and (X[i] + 1 == X[k] or X[i] - 1 == X[k]):
                    Model.add_constr(Mine[i] >= Mine[k])

### Running the model, registering it in a file and returning the results

##### It's worth noting that our results were slightly different from minelib's outcomes. This happened because, in our optimizator, it's utilized a direct precedence rule between the blocks, where a block can only be removed if the 9 or 5 blocks above it are removed as well. Therefore, we're making sure that the slope has, at maximum, 45° inclination in x-y axis in the case of the most conservative, 9 block precedence. However, in the diagonals, the max slope angle has, approximately, 35°. This angle ensures the safety boundaries of the pit, but does not harness the pit's max potential. 
##### Meanwhile, the website's precedence method envolves surface evaluation, which requires a consideratively more complex elaboration compared to what we've used; the exact value, given by 1,415,655,436, represents a relatively low difference from the 1,364,462,123 result in the 9 precedence model: 4%.
##### For the 5 block precedence, which doesn't guarantee the maximum 45º slope, we have an estimated return of 1,442,963,479, about 2% higher than the minelib’s estimate. 
##### As a curiosity, those two average out at 99,2% precision, one being overshot and the other being undershot.

In [ ]:
Result = Model.optimize()
print("Objective Value:", Model.objective_value)
Model.write("Model.lp")

Objective Value: 1442963479.139613


In [ ]:
xtemp = []
ytemp = []
ztemp = []
vm = []

for i in ID:
    if Mine[i].x > 0:
        xtemp.append(X[i])
        ytemp.append(Y[i])
        ztemp.append(Z[i])
        vm.append(Mine[i].x)
data_mine = {'x': xtemp, 'y': ytemp, 'z': ztemp, 'vm': vm}
data_mine = pd.DataFrame(data_mina)

xtemp = []
ytemp = []
ztemp = []
vm = []
for i in list(dataset_minelib['id'].values):
    for j in ID:
      if i == j:
          xtemp.append(X[i])
          ytemp.append(Y[i])
          ztemp.append(Z[i])
          vm.append(1)
dataset_minelib = {'x': xtemp, 'y': ytemp, 'z': ztemp, 'vm': vm}
dataset_minelib = pd.DataFrame(dataset_minelib)


,x,y,z,vm
0,6,28,14,1
1,6,29,14,1
2,6,30,14,1
3,6,31,14,1
4,6,32,14,1
...,...,...,...,...
8511,42,31,13,1
8512,42,32,13,1
8513,42,33,13,1
8514,42,34,13,1


In [ ]:
fig1=px.scatter_3d(dataset,x="x",y="y",z='z',color='proc_profit',
                   color_continuous_scale=px.colors.get_colorscale('thermal'),
                   title="Mine Block Model",range_color=[-6,20],
                   opacity=0.2,width=700,height=700)
fig1.update_traces(marker=dict(symbol='circle',size=4,line=dict(width=0,color='black')))
fig1.show()

In [ ]:
fig2=px.scatter_3d(data_mine,x="x",y="y",z='z',color='vm',
                   color_continuous_scale=px.colors.get_colorscale('thermal'),
                   title="OPTIMIZED MINE BLOCK MODEL",range_color=[0,1],
                   opacity=1, width=700,height=700)
fig2.update_traces(marker=dict(symbol='square',size=5,line=dict(width=1,color='black')))
fig2.show()

In [ ]:
fig3=px.scatter_3d(dataset_minelib,x="x",y="y",z='z',color='vm',title="MINELIB'S BLOCK MODEL",
                   color_continuous_scale=px.colors.get_colorscale('thermal'),
                   range_color=[0,1],opacity=1, width=700,height=700)
fig3.update_traces(marker=dict(symbol='square',size=5,line=dict(width=1,color='black')))
fig3.show()